In [1]:
import subprocess
import os

def convert_to_wav(input_file, output_file='output.wav'):
    # Use FFmpeg to convert to WAV format
    command = ['ffmpeg', '-i', input_file, output_file]
    subprocess.run(command, check=True)
    return output_file


In [2]:
import whisper

model = whisper.load_model("base")

# Convert any audio/video format to WAV
input_file = "C:/Users/Mohamed Walid/OneDrive/Desktop/sample2.flac" 
wav_file = convert_to_wav(input_file)

# Transcribe the WAV file using Whisper
result = model.transcribe(wav_file)

# Print the transcribed text
print("Transcribed Text: ", result['text'])


Transcribed Text:   Before he had time to answer a much encumbered Vera burst into the room with the question, I say, can I leave these here? These were a small black pig and a lusty specimen of black red gamecock.


In [3]:
import os
import subprocess
import whisper
import logging
from typing import Optional, Dict, Union
from pathlib import Path

class WhisperTranscriber:
    """
    A class to handle audio/video transcription using OpenAI's Whisper model.
    Supports multiple input formats and handles conversion to WAV using FFmpeg.
    """
    
    def __init__(self):
        """
        Initialize the transcriber with the 'base' Whisper model.
        """
        self.setup_logging()
        self.model = whisper.load_model("base")
        self.language = "en"

    def setup_logging(self) -> None:
        """Configure logging for the transcriber."""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def convert_to_wav(self, input_file: Union[str, Path], output_dir: Optional[str] = None) -> Optional[str]:
        """
        Convert input audio/video file to WAV format using FFmpeg.
        
        Args:
            input_file (Union[str, Path]): Path to input audio/video file
            output_dir (Optional[str]): Directory for output WAV file
            
        Returns:
            Optional[str]: Path to output WAV file if successful, None otherwise
        """
        try:
            input_path = Path(input_file)
            if not input_path.exists():
                self.logger.error(f"Input file not found: {input_file}")
                return None
            
            # Determine output path
            if output_dir:
                output_path = Path(output_dir) / f"{input_path.stem}.wav"
            else:
                output_path = input_path.with_suffix('.wav')
                
            # Create output directory if it doesn't exist
            output_path.parent.mkdir(parents=True, exist_ok=True)
            
            # FFmpeg command for conversion
            command = [
                'ffmpeg',
                '-i', str(input_path),
                '-ar', '16000',  # Sample rate 16kHz
                '-ac', '1',      # Mono audio
                '-c:a', 'pcm_s16le',  # 16-bit PCM encoding
                str(output_path),
                '-y'  # Overwrite output file if exists
            ]
            
            self.logger.info(f"Converting {input_path} to WAV format")
            result = subprocess.run(
                command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            
            if result.returncode != 0:
                self.logger.error(f"FFmpeg conversion failed: {result.stderr}")
                return None
                
            self.logger.info("Conversion successful")
            return str(output_path)
            
        except Exception as e:
            self.logger.error(f"Error during conversion: {str(e)}")
            return None
    
    def transcribe(self, 
                  input_file: Union[str, Path], 
                  output_dir: Optional[str] = None,
                  cleanup: bool = True) -> Dict:
        """
        Transcribe audio/video file using Whisper.
        
        Args:
            input_file (Union[str, Path]): Path to input audio/video file
            output_dir (Optional[str]): Directory for temporary WAV file
            cleanup (bool): Whether to delete temporary WAV file after transcription
            
        Returns:
            Dict: Transcription result containing text and other metadata
        """
        try:
            # Convert to WAV if input is not already WAV
            input_path = Path(input_file)
            if input_path.suffix.lower() != '.wav':
                self.logger.info("Converting input file to WAV format")
                wav_file = self.convert_to_wav(input_file, output_dir)
                if not wav_file:
                    raise RuntimeError("Failed to convert input file to WAV format")
            else:
                wav_file = str(input_path)
            
            # Perform transcription
            self.logger.info("Starting transcription")
            result = self.model.transcribe(wav_file, language=self.language)
            self.logger.info("Transcription completed successfully")
            
            # Cleanup temporary WAV file if requested
            if cleanup and input_path.suffix.lower() != '.wav':
                try:
                    os.remove(wav_file)
                    self.logger.info(f"Cleaned up temporary WAV file: {wav_file}")
                except Exception as e:
                    self.logger.warning(f"Failed to cleanup temporary file: {str(e)}")
            
            return result
            
        except Exception as e:
            self.logger.error(f"Transcription failed: {str(e)}")
            raise

In [7]:
# Initialize transcriber
transcriber = WhisperTranscriber()
    
input_file = "C:/Users/Mohamed Walid/OneDrive/Desktop/Small Talk  Everyday English.mp4"
try:
    result = transcriber.transcribe(input_file)
        
    # Store the transcribed text in a variable
    transcribed_text = result['text']
        
    # Print the transcribed text
    print(f"Transcription: {transcribed_text}")
        
    # You can now use transcribed_text for further processing
    # For example, save to a file or use in other functions
except Exception as e:
    print(f"Transcription failed: {e}")

2024-10-27 01:12:31,728 - INFO - Converting input file to WAV format
2024-10-27 01:12:31,730 - INFO - Converting C:\Users\Mohamed Walid\OneDrive\Desktop\Small Talk  Everyday English.mp4 to WAV format
2024-10-27 01:12:31,937 - INFO - Conversion successful
2024-10-27 01:12:31,938 - INFO - Starting transcription
2024-10-27 01:12:38,762 - INFO - Transcription completed successfully
2024-10-27 01:12:38,764 - INFO - Cleaned up temporary WAV file: C:\Users\Mohamed Walid\OneDrive\Desktop\Small Talk  Everyday English.wav


Transcription:  So, what's new Mark? How is your new job going? To be honest, I can't complain. I really love the company that I am working for. My co-workers are all really friendly and helpful. They really help me feel welcome. It's a really energetic and fun atmosphere. My boss is hilarious and he's really flexible. Really? How so? He allows me to come in when I want and make my own hours. I can also leave early if I start early. There is no real dress code either. I can wear jeans and a t-shirt if I want. I can even wear shorts in the summer. Wow. It sounds really cool. I can't stand wearing a suit every day. Which do you prefer? Working late or finishing early? I prefer finishing early. I really enjoy the morning. I love getting up early and going for a run. There is nothing like watching the sunrise while drinking my morning coffee. Really? I am opposite. I love sleeping in. I am most alert in the evenings. I'm a real night owl. Well, you know what they say. The early bird catche

In [8]:
transcribed_text

" So, what's new Mark? How is your new job going? To be honest, I can't complain. I really love the company that I am working for. My co-workers are all really friendly and helpful. They really help me feel welcome. It's a really energetic and fun atmosphere. My boss is hilarious and he's really flexible. Really? How so? He allows me to come in when I want and make my own hours. I can also leave early if I start early. There is no real dress code either. I can wear jeans and a t-shirt if I want. I can even wear shorts in the summer. Wow. It sounds really cool. I can't stand wearing a suit every day. Which do you prefer? Working late or finishing early? I prefer finishing early. I really enjoy the morning. I love getting up early and going for a run. There is nothing like watching the sunrise while drinking my morning coffee. Really? I am opposite. I love sleeping in. I am most alert in the evenings. I'm a real night owl. Well, you know what they say. The early bird catches the worm. Yo